In [ ]:
%matplotlib nbagg
import sys
import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
#import instrument
import qcodes.instrument_drivers.rohde_schwarz.ZNB as ZNB

from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id, load_by_id
from qcodes import new_experiment, Station, Instrument
from qcodes.dataset.data_export import get_data_by_id
from qcodes import Parameter

In [ ]:
import reload
reload(qcodes)

# initialize database

In [83]:
#initialize database
import datetime
dataset_name = 'ABS_W1_3_RES'
qc.config["core"]["db_location"] = datetime.date.today().strftime(r'D:\OneDrive\BF3\Data\%Y%m%d_'+dataset_name+'.db')
# automatically uses a different DB file for each month
qc.dataset.database.initialise_database()
qc.config.core.db_location

Upgrading database: : 0it [00:00, ?it/s]


'D:\\OneDrive\\BF3\\Data\\20190630_ABS_W1_3_RES.db'

In [84]:
station = qc.Station()

In [85]:
exp = qc.new_experiment("Andreev spectroscopy", sample_name="ABS W1 #3 RES")

# Bluefors fridge log tunnel

## Bluefors log opening


In [ ]:
#set the path here of the shared desktop file
BF_PC_PATH = '\\\\MININT-ETA6HHA\\Desktop\\BF Log Files\\' 

import time

In [ ]:
logfile_name = 'CH5 T'
days_in_the_past = 0
ts = datetime.datetime.today() - datetime.timedelta(days=days_in_the_past)
ds = '%02d-%02d-%02d'%(ts.year-2000, ts.month, ts.day) 
pth = BF_PC_PATH + ds + '\\' + logfile_name+ ' ' + ds+'.log'

T = np.loadtxt(pth, usecols = (2), delimiter = ',')
plt.figure()
plt.plot(T)
plt.title(ds + ' ' + pth)
plt.xlim(18000, 22000)
logfile_name = 'CH9 T'
ts = datetime.datetime.today() - datetime.timedelta(days=days_in_the_past)
ds = '%02d-%02d-%02d'%(ts.year-2000, ts.month, ts.day) 
pth = BF_PC_PATH + ds + '\\' + logfile_name+ ' ' + ds+'.log'
T1 = T
T = np.loadtxt(pth, usecols = (2), delimiter = ',')
#plt.figure()
plt.plot(T)
plt.title(pth)
plt.xlim(18000,22000)
plt.ylim((0,2))

plt.figure()
plt.plot(T1-T)
plt.title(pth)
plt.xlim(18000,22000)
plt.ylim((-1,1))

In [ ]:
import csv
BF_PC_PATH = '\\\\MININT-ETA6HHA\\Desktop\\BF Log Files\\' 

#code to get the timestamp copied here from the thermometer class

logfile_name = 'maxigauge'
days_in_the_past = 0
ts = datetime.datetime.today() - datetime.timedelta(days=days_in_the_past)
ds = '%02d-%02d-%02d'%(ts.year-2000, ts.month, ts.day) 
pth = BF_PC_PATH + ds + '\\' + logfile_name+ ' ' + ds+'.log'

file_list = []
with open(pth) as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        #print(row)
        file_list.append(row)
csvFile.close()

#
print(file_list[0])
times = [file_list[i][1] for i in range(len(file_list))]
Ps = []
for k in range(0,6):
    Ps.append(np.array([float(file_list[i][5+k*6]) for i in range(len(file_list))]))

i_start = 0
i_stop = -1
for k in range(len(Ps)):
    plt.figure()
    plt.plot(times[i_start:i_stop], Ps[k][i_start:i_stop])
    plt.ylabel('Pressure (mBar)')
    plt.xlabel('Timestamp')
    plt.title('P%d at ' % (k+1) + ds )
    

## Thermometer using BF logs

In [ ]:
#set the path here of the shared desktop file
from pytopo.sweep import sweep, measure, nest, chain, szip, getter, setter, time_trace, do_experiment
from pytopo.qctools.instruments import create_inst

class Thermometer(Instrument):
    
    def __init__(self, name, logfile_path, **kw):
        super().__init__(name, **kw)        
        self.add_parameter('temperature', get_cmd=self._get_temperature, set_cmd=None, unit='K')
        
        self.add_parameter('counter', get_cmd=self._counter, set_cmd=None, unit='')
        self._ind = 0
        print('Path of network drive(set this manually dear USER): ', logfile_path)
        self.logfile_path = logfile_path
        
    def _counter(self):
        self._ind += 1
        return self._ind
    def _get_temperature(self):
        ts = datetime.datetime.today()
        ds = '%02d-%02d-%02d'%(ts.year-2000, ts.month, ts.day)
        pth = self.logfile_path + ds + '\\CH9 T '+ds+'.log'
        time.sleep(0.1)
        try:
            T = np.loadtxt(pth, usecols = (2), delimiter = ',')[-1]
        except: # in the first few seconds of a day, a new log file hasn't been created
            ts = datetime.datetime.today() - datetime.timedelta(days=1) # use the last entry in the previous day
            ds = '%02d-%02d-%02d'%(ts.year-2000, ts.month, ts.day)
            pth = self.logfile_path + ds + '\\CH9 T '+ds+'.log'
            T = np.loadtxt(pth, usecols = (2), delimiter = ',')[-1]
        return T
    
        
    
T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
T_probe.temperature()



In [ ]:
BF_PC_PATH = '\\\\MININT-ETA6HHA\\Desktop\\BF Log Files\\'
ds = '%02d-%02d-%02d'%(ts.year-2000, ts.month, ts.day)
pth = BF_PC_PATH+ ds + '\\CH9 T '+ds+'.log'

# VNA measurements of lines.
https://github.com/QCoDeS/Qcodes/blob/master/docs/examples/driver_examples/Qcodes%20example%20with%20Rohde%20Schwarz%20ZNB.ipynb 

In [ ]:
vna = create_inst(ZNB.ZNB, "VNA", 'USB0::0x0AAD::0x01A1::101879::INSTR', force_new_instance=True, init_s_params=False)
#station.remove_component(vna)
station.add_component(vna)
# the channel we want to measure. we set init_s_params false to not have to measure all channels, thus saving time 
channel_name = 'S21' ##Now we add a channel S12: Since the HEMT is connected to port 1. 
vna.add_channel(channel_name)


In [ ]:
 meas = Measurement()

In [ ]:
import time
import numpy as np
def do_single_sweep(n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, power=-60, n_avg=10, channel_name='S12',
                   extra_params = []):
    """
    extra_params expects [(qcodes.Parameter, value)...]
    """

    # let's set the trace properties. 
    trace = getattr(vna.channels, channel_name)
    trace.avg(n_avg)
    trace.npts(n_pts)
    trace.power(power)
    trace.start(f_start)
    trace.stop(f_stop)
    vna.rf_on()
    
    
    #now setup the qcodes measurement object
    meas = Measurement()
    #meas.register_parameter(temperature, setpoints=['time',])
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter('time', unit='s', paramtype='numeric')
    meas.register_custom_parameter('power', unit='dBm', setpoints=['time'])
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                    setpoints=['frequency'],
                                   paramtype='array'),
    meas.register_custom_parameter('magnitude_raw', unit='V', 
                                   setpoints=['frequency'],
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=[ 'frequency'],
                                   paramtype='array')
    
    #any custom params that someone wants to save as well in the measuremebt
    for param in extra_params:
        meas.register_parameter(param[0])
    
    with meas.run() as datasaver:
        fvals = np.linspace(trace.start(), trace.stop(), trace.npts())
        mag, phase = trace.trace_mag_phase()
        mag_db = 20*np.log10(mag) #convert to dB
        result = [ ('time', time.time()),
           # (temperature, temperature()),
           # (temperature_sample, temperature_sample()),
            ('frequency', fvals),
            ('magnitude_raw', mag),
            ('magnitude_dB',mag_db ),
            ('phase', phase),
            ('power', power)]
        result.extend(extra_params)
        datasaver.add_result(
            *result
           )
    return datasaver.run_id


def do_single_sweep_with_temperature(n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, power=-60, n_avg=10, channel_name='S12'):

    """
    Saves a timestamp and a temperature. 
    """
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    trace = getattr(vna.channels, channel_name)
    trace.avg(n_avg)
    trace.npts(n_pts)
    trace.power(power)
    trace.start(f_start)
    trace.stop(f_stop)
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    #meas.register_parameter(temperature, setpoints=['time',])
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter('time', unit='s', paramtype='numeric')
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric', setpoints=['time'])
    meas.register_custom_parameter('power', unit='dBm', setpoints=['time'])
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                    setpoints=['frequency'],
                                   paramtype='array'),
    meas.register_custom_parameter('magnitude_raw', unit='V', 
                                   setpoints=['frequency'],
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=[ 'frequency'],
                                   paramtype='array')
    
    with meas.run() as datasaver:
        fvals = np.linspace(trace.start(), trace.stop(), trace.npts())
        mag, phase = trace.trace_mag_phase()
        mag_db = 20*np.log10(mag) #convert to dB
        result = [ ('time', time.time()),
           # (temperature, temperature()),
           # (temperature_sample, temperature_sample()),
            ('frequency', fvals),
            ('magnitude_raw', mag),
            ('magnitude_dB',mag_db ),
            ('phase', phase),
            ('power', power),
            ('temperature', temperature)]
        datasaver.add_result(
            *result
           )
    return datasaver.run_id

In [ ]:
power_levels = [-20]
#res_freqs = [3.535e9, 4.020e9, 4.847e9, 5.889e9]
res_freqs = [3.6348e9, 3.9983e9, 4.6754e9, 5.6459e9]
df = 0.04e9
channel_name = 'S21'

for power in power_levels:   
    for freq in res_freqs:
        print(freq-df)
        print(freq+df)
        print(power)
        run_id = do_single_sweep_with_temperature(n_pts = 30000, f_start = freq-df, f_stop = freq+df,
                                                  power=power, n_avg=100, channel_name=channel_name)
        
        
        


In [ ]:
run_id = do_single_sweep_with_temperature(n_pts = 30000, f_start = 3.3e9, f_stop = 7e9,
                                                  power=-20, n_avg= 100, channel_name='S21')

In [ ]:
aves, bla = plot_by_id(run_id)

In [ ]:
run_id=5
dat=load_by_id(run_id)
freqs = np.squeeze(dat.get_data('frequency'))
mag_db = np.squeeze(dat.get_data('magnitude_dB'))

plt.figure()
plt.plot(freqs,mag_db)
plt.grid()
plt.title('Run id: %d %s' %(run_id, qc.config["core"]["db_location"]))
plt.axhline(y = np.max(mag_db), label='Max S21=%.3f dB' % np.max(mag_db), color='g')
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
#plt.xlim(4e9,8e9)

In [ ]:
vna.rf_off()

In [ ]:
meas = Measurement()
vna.channels.S12.format('dB')
meas.register_parameter(vna.channels.S12.trace)
with meas.run() as datasaver:
    get_v = vna.channels.S12.trace.get()
    datasaver.add_result((vna.channels.S12.trace, get_v))
ax, cbax = plot_by_id(datasaver.run_id)

## 2 D sweeps: power 

In [ ]:
def do_2d_power_sweep(powers=[],n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, n_avg=10, channel_name='S12'):

    """
. 
    """
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    channel = getattr(vna.channels, channel_name)
    
    channel.avg(n_avg)
    channel.npts(n_pts)
    channel.start(f_start)
    channel.stop(f_stop)
    channel.power(powers[0])
    channel.format('Complex')
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter('power', unit='dBm'),
    meas.register_custom_parameter('Re_S', unit='V', 
                                    setpoints=['frequency','power'],
                                   paramtype='array'),
    meas.register_custom_parameter('Im_S', unit='V', 
                                   setpoints=['frequency', 'power'],
                                   paramtype='array')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=[ 'frequency', 'power'],
                                   paramtype='array')
    
    with meas.run() as datasaver:
        for power in powers:
            channel.power(power)
            fvals = np.linspace(channel.start(), channel.stop(), channel.npts())
            S12 = channel.trace.get()
            mag_db = 20*np.log10(np.abs(S12)) #convert to dB
            result = [ 
                ('frequency', fvals),
                ('Re_S', np.real(S12)),
                ('magnitude_dB',mag_db ),
                ('Im_S', np.imag(S12)),
                ('power', power),
                ('temperature', temperature)]
            datasaver.add_result(
                *result
               )
            print('done power %d' % power)
    return datasaver.run_id




In [118]:
set_yoko_field(0)

In [130]:
res_freqs = np.array([3.9983e9, 3.6348e9, 4.6754e9 ,5.6459e9])
df = 20e6
  
for freq in res_freqs:
    print(freq-df)
    print(freq+df)
    run_id_1 = do_2d_power_sweep(powers=np.arange(-40,10,5),n_pts = 10000, f_start = freq-df, f_stop = freq+df, n_avg=300, channel_name='S21')
vna.rf_off()

3978300000.0
4018300000.0
Instrument T_probe already exists.
Path of network drive(set this manually dear USER):  \\MININT-ETA6HHA\Desktop\BF Log Files\
Starting experimental run with id: 60


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


done power -40


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


done power -35


...

done power -5


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


done power 0


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


done power 5


In [ ]:
data = load_by_id(23)
freqs = np.squeeze(np.array(data.get_data('frequency')))
powers = np.squeeze(np.array(data.get_data('power')))
mag = np.squeeze(np.array(data.get_data('magnitude_dB')))
plt.figure()
plt.plot(freqs.T,mag.T, 'x')

In [ ]:
T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
temperature = T_probe.temperature()
print(temperature)

In [131]:
vna.rf_off()

VisaIOError: ('VI_ERROR_SYSTEM_ERROR (-1073807360): Unknown system error (miscellaneous error).', "writing 'OUTP1 OFF' to <ZNB: VNA>")

In [ ]:
do_2d_power_sweep(powers=np.arange(-40,0, 5),n_pts = 30000, f_start = 2.9e9, f_stop = 12e9, n_avg=50, channel_name='S12')

## Temperature sweep

In [ ]:
def do_2d_T_sweep(T_max = 20, power = -20,n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, n_avg=10, channel_name='S12'):

    """
. 
    """

    # let's set the trace properties. 
    channel = getattr(vna.channels, channel_name)
    
    channel.avg(n_avg)
    channel.npts(n_pts)
    channel.start(f_start)
    channel.stop(f_stop)
    channel.power(power)
    channel.format('Complex')
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter('power', unit='dBm'),
    meas.register_custom_parameter('Re_S', unit='V', 
                                    setpoints=['frequency','temperature'],
                                   paramtype='array'),
    meas.register_custom_parameter('Im_S', unit='V', 
                                   setpoints=['frequency', 'temperature'],
                                   paramtype='array')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=[ 'frequency', 'temperature'],
                                   paramtype='array')
    
    with meas.run() as datasaver:
      #first try to get temperature:
        T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
        temperature = T_probe.temperature()
        while temperature <  T_max:
            T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
            temperature = T_probe.temperature()
            
            fvals = np.linspace(channel.start(), channel.stop(), channel.npts())
            S12 = channel.trace.get()
            mag_db = 20*np.log10(np.abs(S12)) #convert to dB
            result = [ 
                ('frequency', fvals),
                ('Re_S', np.real(S12)),
                ('magnitude_dB',mag_db ),
                ('Im_S', np.imag(S12)),
                ('power', power),
                ('temperature', temperature)]
            datasaver.add_result(
                *result
               )
            print('done T=%.3f' % temperature)
    return datasaver.run_id


def do_2d_T_sweep_cooldown(T_min = 0.003, power = -20,n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, n_avg=10, channel_name='S12'):

    """
. 
    """

    # let's set the trace properties. 
    channel = getattr(vna.channels, channel_name)
    
    channel.avg(n_avg)
    channel.npts(n_pts)
    channel.start(f_start)
    channel.stop(f_stop)
    channel.power(power)
    channel.format('Complex')
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter('power', unit='dBm'),
    meas.register_custom_parameter('Re_S', unit='V', 
                                    setpoints=['frequency','temperature'],
                                   paramtype='array'),
    meas.register_custom_parameter('Im_S', unit='V', 
                                   setpoints=['frequency', 'temperature'],
                                   paramtype='array')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=[ 'frequency', 'temperature'],
                                   paramtype='array')
    
    with meas.run() as datasaver:
      #first try to get temperature:
        T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
        temperature = T_probe.temperature()
        while temperature >  T_min:
            T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
            temperature = T_probe.temperature()
            fvals = np.linspace(channel.start(), channel.stop(), channel.npts())
            S12 = channel.trace.get()
            mag_db = 20*np.log10(np.abs(S12)) #convert to dB
            result = [ 
                ('frequency', fvals),
                ('Re_S', np.real(S12)),
                ('magnitude_dB',mag_db ),
                ('Im_S', np.imag(S12)),
                ('power', power),
                ('temperature', temperature)]
            datasaver.add_result(
                *result
               )
            print('done T=%.3f' % temperature)
    return datasaver.run_id



In [ ]:
do_2d_T_sweep_cooldown(power=10,T_min=0.03, n_pts = 10000, f_start = 2.5e9, f_stop = 10e9, n_avg=5, channel_name='S21')

# Magnet 

Check that the magnet is the "right" one and so the parameters (coil constant and so on...)

In [ ]:
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430,AMI430_3D
from pytopo.qctools import instruments as instools


In [ ]:
ami_x = instools.create_inst(AMI430, "AMI430_x", "169.254.178.28", port = 7180, has_current_rating=True)
ami_y = instools.create_inst(AMI430, "AMI430_y", "169.254.175.39", port = 7180, has_current_rating=True)
ami_z = instools.create_inst(AMI430, "AMI430_z", "169.254.137.107", port = 7180, has_current_rating=True)

ramp_limit_multiplier = 0.6
ami_x.current_ramp_limit(0.1805*ramp_limit_multiplier) #for QT3 magnet 0.1802
ami_x.coil_constant(0.01464)  #for QT3 magnet 0.01466
ami_x.current_rating(68.31)  #for QT3 magnet 68.21

ami_y.current_ramp_limit(0.1810*ramp_limit_multiplier) #for QT3 magnet 0.1784
ami_y.coil_constant(0.01477) #for QT3 magnet 0.01464
ami_y.current_rating(67.7) #for QT3 magnet 68.31

ami_z.current_ramp_limit(0.0760*ramp_limit_multiplier) #for QT3 magnet 0.0775
ami_z.coil_constant(0.0836)  #for QT3 magnet 0.0818
ami_z.current_rating(71.77)  #for QT3 magnet 73.35

#ami = instools.create_inst(AMI430_3D, "AMI430", ami_x, ami_y, ami_z, 2)
#ami_3d = AMI430_3D("AMI_3D", ami_x, ami_y, ami_z, 1);

In [ ]:
print(ami_x.ramp_rate(), 'T/ ramp rate unit')
#print(ami_y.ramp_rate())
print(ami_z.ramp_rate())
print(ami_x.ramp_rate_units())
print(ami_z.current_ramp_limit())
#30.1802*ramp_limit_multiplier
print(ami_x.field())
print(ami_y.field())
print(ami_z.field())
#print (ami_3d.field_measured())
#print(ami_3d.phi_measured())

In [ ]:
ami_y.ramp_rate(0.1e-3)

In [ ]:
#note that 0.01 mT is already fluctuating between 
get_yoko_field()

In [ ]:

print(ami_y.field())

## Yoko current source for y axis

In [ ]:
from qcodes.instrument_drivers.yokogawa.GS200 import GS200
yoko1 = create_inst(GS200,"yoko1", "USB0::0x0B21::0x0039::91T926460::INSTR", force_new_instance=True)
station.add_component(yoko1)
#yoko1.off()

In [135]:
yoko1.off()

In [ ]:
yoko1.current_limit(200e-3)
yoko1.source_mode("CURR")
yoko1.on()
yoko1.auto_range(True)
yoko1.current()
#yoko1.current(0)


In [100]:
def set_yoko_field(field_strength, current_rating= 67.7, #QT2 current rating y-axis magnet. 
                  yoko_ramp_step = 1e-6, # in A
                  yoko_ramp_delay = 1e-6): # in s):
    """
    goes as 67.7A/T
    """
    amp = field_strength * current_rating
    if(np.abs(amp) > 200e-3):
        raise Exception("Required current for field beyond yoko limit")
    
    yoko1.ramp_current(amp, yoko_ramp_step, yoko_ramp_delay)
def get_yoko_field(current_rating= 67.7):
    """
    returns the field in Tesla
    """
    return yoko1.current() / current_rating 
print(get_yoko_field())


class YokoMagnet(Instrument):
    """
    Just a class to do 3D magnet stuff but then with the yoko as y-axis instead of ami-y
    """    
    def __init__(self, name, ami_x, ami_z, **kwargs):
        super().__init__(name, **kwargs)
        
        #use the ami x and z params
        self.add_parameter('z', get_cmd=ami_z.field, set_cmd=ami_z.field, unit='T')
        self.add_parameter('x', get_cmd=ami_x.field, set_cmd=ami_x.field, unit='T')
        #but define custom function for y
        self.add_parameter('y', get_cmd=get_yoko_field, set_cmd=set_yoko_field, unit='T')
    
    def transform_spherical_to_cartesian(self, r, theta, phi):
        """
        Does what it sais, uses the same convention as FieldVector QCodes math helper function and thus as the AMI_3D driver
        note radians...
        """
        z = r * np.cos(theta)
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        return x, y, z

5.559527326440177e-05


In [ ]:
#yoko_magnet = create_inst(YokoMagnet, 'yoko_magnet',ami_x = ami_x,ami_z = ami_z, force_new_instance=True)
#station.add_component(yoko_magnet)

In [133]:
set_yoko_field(0)

In [134]:
get_yoko_field()

0.0

In [77]:
ami_z.field(0)

In [ ]:
yoko_magnet.z(0)
yoko_magnet.x(0)
yoko_magnet.y(0)

In [76]:
res_freqs

array([4.6704e+09, 5.6409e+09, 3.6298e+09, 3.9933e+09])

In [87]:
ami_z.field(0)
### b_min = 0
b_max = 0.5e-3
b_min = 0
b_step = 0.5e-6
res_freqs = np.array([ 4.6754e9 5.6459e9, 3.6348e9,3.9983e9])

y_fields = np.arange(b_min, b_max, b_step)

for f0 in res_freqs:
    #f0 = res_freqs[2]
    df = 5e6

    power=-40
    n_pts = 200
    f_start = f0-df
    f_stop = f0+df
    n_avg=1000

    channel_name='S21'


    #get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()

    # set VNA stuff
    vna_channel = getattr(vna.channels, channel_name)

    vna_channel.avg(n_avg)
    vna_channel.npts(n_pts)
    vna_channel.start(f_start)
    vna_channel.stop(f_stop)
    vna_channel.power(power)
    vna_channel.format('Complex')
    vna.rf_on()


    #START OF THE MSM
    #now setup the qcodes measurement object
    meas = Measurement()

    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')

    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')

    meas.register_custom_parameter('B_y', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_x', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_z', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_y_meas', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_x_meas', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_z_meas', unit='T', paramtype='numeric')


    # set vna props
    vna_trace_setpoints = ['frequency', 'B_y']

    meas.register_custom_parameter('power', unit='dBm'),

    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=vna_trace_setpoints,
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='V', 
                                   setpoints=vna_trace_setpoints,
                                   paramtype='array')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=vna_trace_setpoints,
                                   paramtype='array')


    with meas.run() as datasaver:
        for field in y_fields:
            #set the B field
            print("Setting Bfield to %.2e" % field)
            set_yoko_field(field)

            #VNA stuff
            fvals = np.linspace(vna_channel.start(), vna_channel.stop(), vna_channel.npts()) 
            S12 =vna_channel.trace.get()
            mag_db = 20*np.log10(np.abs(S12)) #convert to dB
            result = [ 
                ('frequency', fvals),
                ('magnitude_dB',mag_db ),
                ('phase', np.angle(S12)),
                ('mag', np.abs(S12)),
                ('power', power),
                ('temperature', temperature)]

            #save the measured fields
            #result.append(('B_y_meas', ami_y.field()))
            #result.append(('B_x_meas', ami_x.field()))
            #result.append(('B_z_meas', ami_z.field()))
            #save the target field
            result.append(('B_y', field))

            run_id = datasaver.add_result(
                *result
               )
    print(run_id)

Instrument T_probe already exists.
Path of network drive(set this manually dear USER):  \\MININT-ETA6HHA\Desktop\BF Log Files\
Starting experimental run with id: 2
Setting Bfield to 0.00e+00


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


Setting Bfield to 5.00e-07


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


Setting Bfield to 1.00e-06


...

QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


Setting Bfield to 3.24e-04


QCoDeS Dataset does not currently support Complex values. Will discard the imaginary part. In order to acquire phase and amplitude use the FrequencySweepMagPhase parameter.


Setting Bfield to 3.24e-04


KeyboardInterrupt: 

## 2D gate frequency sweeps

In [ ]:
def do_gate_resonator_sweep(gate_channel, gate_voltages,gate_name='V_g',
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=-30, n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, n_avg=10, channel_name='S12'):

    """
   
    """
    #check the gate voltage
    print("Starting voltage", gate_channel.voltage())
    
    
    
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    vna_channel = getattr(vna.channels, channel_name)
    
    vna_channel.avg(n_avg)
    vna_channel.npts(n_pts)
    vna_channel.start(f_start)
    vna_channel.stop(f_stop)
    vna_channel.power(power)
    vna_channel.format('Complex')
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter(gate_name, unit='V'),
    meas.register_custom_parameter('power', unit='dBm'),
    meas.register_custom_parameter('Re_S', unit='V', 
                                    setpoints=['frequency',gate_name],
                                   paramtype='array'),
    meas.register_custom_parameter('Im_S', unit='V', 
                                   setpoints=['frequency', gate_name],
                                   paramtype='array')
    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=['frequency', gate_name],
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='V', 
                                   setpoints=['frequency', gate_name],
                                   paramtype='array')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=[ 'frequency', gate_name],
                                   paramtype='array')
    
    with meas.run() as datasaver:
        for gate_voltage in gate_voltages:
            
            #ramp the gate voltage
            print("Ramping %s to %.2e" % (gate_name, gate_voltage))
            ramp(gate_channel.voltage, gate_voltage, gate_ramp_stepsize, gate_ramp_delay )
            print("Done ramping")
            fvals = np.linspace(vna_channel.start(), vna_channel.stop(), vna_channel.npts())
            S12 = vna_channel.trace.get()
            mag_db = 20*np.log10(np.abs(S12)) #convert to dB
            result = [ 
                ('frequency', fvals),
                ('Re_S', np.real(S12)),
                ('magnitude_dB',mag_db ),
                ('Im_S', np.imag(S12)),
                ('phase', np.angle(S12)),
                ('mag', np.abs(S12)),
                ('power', power),
                (gate_name, gate_voltage),
                ('temperature', temperature)]
            datasaver.add_result(
                *result
               )
            print('done power %d' % power)
    return datasaver.run_id




def do_multiple_gate_resonator_sweep(
                            gate_channel, gate_voltages,gate_name='V_g',
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=-30, n_pts = 10000, f_start = 3.2e9, f_stop = 6.5e9, n_avg=10, channel_name='S12'):

    """
   
    """
    if not isinstance(gate_channel, list):
        gate_channel = [gate_channel] #allow for multple gates to be swept at once. 
        if not isinstance(gate_name, list):
            gate_name = [gate_name]
        else:
            raise Exception('Either give both gate channel and gate name as a list or both as a value..')
    
    #check the gate voltage
    for chan in gate_channel:
        print(chan,chan.voltage())
    
    #set the gate name to be all gates together
    long_gate_name = ''
    for name in gate_name:
        long_gate_name += name + '_' #make one variable out of multiple gates snce they are swept together
    
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    vna_channel = getattr(vna.channels, channel_name)
    
    vna_channel.avg(n_avg)
    vna_channel.npts(n_pts)
    vna_channel.start(f_start)
    vna_channel.stop(f_stop)
    vna_channel.power(power)
    vna_channel.format('Complex')
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency', unit='Hz',
                                   paramtype='array')
    meas.register_custom_parameter(long_gate_name, unit='V'),
    meas.register_custom_parameter('power', unit='dBm'),
    meas.register_custom_parameter('Re_S', unit='V', 
                                    setpoints=['frequency',long_gate_name],
                                   paramtype='array'),
    meas.register_custom_parameter('Im_S', unit='V', 
                                   setpoints=['frequency', long_gate_name],
                                   paramtype='array')
    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=['frequency', long_gate_name],
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='V', 
                                   setpoints=['frequency', long_gate_name],
                                   paramtype='array')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=[ 'frequency', long_gate_name],
                                   paramtype='array')
    
    with meas.run() as datasaver:
        for gate_voltage in gate_voltages:
            
            #ramp the gate voltage
            for i in range(len(gate_channel)):
                print("Ramping %s to %.2e" % (gate_name[i], gate_voltage))
                ramp(gate_channel[i].voltage, gate_voltage, gate_ramp_stepsize, gate_ramp_delay )
                print("Done ramping")
            fvals = np.linspace(vna_channel.start(), vna_channel.stop(), vna_channel.npts())
            S12 = vna_channel.trace.get()
            mag_db = 20*np.log10(np.abs(S12)) #convert to dB
            result = [ 
                ('frequency', fvals),
                ('Re_S', np.real(S12)),
                ('magnitude_dB',mag_db ),
                ('Im_S', np.imag(S12)),
                ('phase', np.angle(S12)),
                ('mag', np.abs(S12)),
                ('power', power),
                (long_gate_name, gate_voltage),
                ('temperature', temperature)]
            datasaver.add_result(
                *result
               )
            print('done power %d' % power)
    return datasaver.run_id


def do_multiple_gate_multiple_resonator_sweep(
                            gate_channel, gate_voltages,gate_name='V_g',
                            f_starts = [3.2e9], f_stops = [6.5e9], f_names=['A'],
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=-30, n_pts = 10000, n_avg=10, channel_name='S12'):

    """
   
    """
    if not isinstance(gate_channel, list):
        gate_channel = [gate_channel] #allow for multple gates to be swept at once. 
        if not isinstance(gate_name, list):
            gate_name = [gate_name]
        else:
            raise Exception('Either give both gate channel and gate name as a list or both as a value..')
    
    #check the gate voltage
    for chan in gate_channel:
        print(chan,chan.voltage())
    
    #set the gate name to be all gates together
    long_gate_name = ''
    for name in gate_name:
        long_gate_name += name + '_' #make one variable out of multiple gates snce they are swept together
    
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    vna_channel = getattr(vna.channels, channel_name)
    
    vna_channel.avg(n_avg)
    vna_channel.npts(n_pts)
    vna_channel.start(f_starts[0])
    vna_channel.stop(f_stops[0])
    vna_channel.power(power)
    vna_channel.format('Complex')
    vna.rf_on()
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    
    meas.register_custom_parameter(long_gate_name, unit='V'),
    meas.register_custom_parameter('power', unit='dBm'),
    
    for i in range(len(fstarts)):
        meas.register_custom_parameter('frequency_'+f_names[i], unit='Hz',
                                   paramtype='array')
        meas.register_custom_parameter('mag_'+f_names[i], unit='V', 
                                       setpoints=['frequency_'+f_names[i], long_gate_name],
                                       paramtype='array')
        meas.register_custom_parameter('phase_'+f_names[i], unit='V', 
                                       setpoints=['frequency_'+f_names[i], long_gate_name],
                                       paramtype='array')
        meas.register_custom_parameter('magnitude_dB_'+f_names[i], unit='dB', 
                                       setpoints=[ 'frequency_'+f_names[i], long_gate_name],
                                       paramtype='array')
    
    with meas.run() as datasaver:
        for gate_voltage in gate_voltages:
            
            #ramp the gate voltage
            for i in range(len(gate_channel)):
                print("Ramping %s to %.2e" % (gate_name[i], gate_voltage))
                ramp(gate_channel[i].voltage, gate_voltage, gate_ramp_stepsize, gate_ramp_delay )
                print("Done ramping")
            
            result = [ 
                    ('power', power),
                    (long_gate_name, gate_voltage),
                    ('temperature', temperature)]
            
            for i in range(len(fstarts)): 
                vna_channel.start(f_starts[i])
                vna_channel.stop(f_stops[i])
                fvals = np.linspace(vna_channel.start(), vna_channel.stop(), vna_channel.npts())
                S12 = vna_channel.trace.get()
                mag_db = 20*np.log10(np.abs(S12)) #convert to dB
                result.extend([
                    ('frequency_'+f_names[i], fvals),
                    ('magnitude_dB_'+f_names[i],mag_db ),
                    ('phase_'+f_names[i], np.angle(S12)),
                    ('mag_'+f_names[i], np.abs(S12)),
                ])
               
                
            datasaver.add_result(
                *result
               )
            print('done power %d' % power)
    return datasaver.run_id


In [ ]:
list(MDAC_channels['A'].values())

### Multiple gates at the same time frequency sweep

In [ ]:
power = -33
n_pts = 200
n_avg = 400
f = f0s[3]
df = 2.5e6
resonator='A'
gate_voltages = np.arange(2.65,2.3,-0.003)
#set the Bfield. 
#set_yoko_field(0.2e-3)

exp = qc.new_experiment("Multi-gate-freq sweep B=%.4e npts=%d n_avg=%d power=%d" %(get_yoko_field(),n_pts, n_avg,power)  , sample_name="abs_w1_5_1_1")

ramp_back_gates = False

#ramp all gates of resonator A
gate_channels =list(MDAC_channels[resonator].values())
gate_names = list(MDAC_channels[resonator].keys())

if ramp_back_gates:
    print('ramping back gates')
    for gate in gate_names:
        ramp(MDAC_channels[resonator][gate].voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

t0 = time.time()
#now do the real thing
do_multiple_gate_resonator_sweep(gate_channels, gate_voltages,gate_names,
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=power, n_pts = n_pts, f_start = f-df, f_stop = f+df, n_avg=n_avg, channel_name='S21')

print(" Time elapsed: %.4f" % (time.time()-t0))

In [ ]:
vna.S21.bandwidth(1000)

## multiple gates multiple res sweep

In [ ]:

power = -30
n_pts = 100
n_avg = 50
df = 5e6
fstarts=np.array(f0s)-df
fstops=np.array(f0s)+df
f_names=['D','C','A','B']
resonator='A' # for gate
gate_voltages = np.arange(-4,4,0.002)
#set the Bfield. 
set_yoko_field(0.2e-3)

exp = qc.new_experiment("Multi-res-gate-freq sweep B=%.4e npts=%d n_avg=%d power=%d" %(get_yoko_field(),n_pts, n_avg,power)  , sample_name="abs_w1_5_1_1")

ramp_back_gates = False

#ramp all gates of resonator A
gate_channels =list(MDAC_channels[resonator].values())
gate_names = list(MDAC_channels[resonator].keys())

if ramp_back_gates:
    print('ramping back gates')
    for gate in gate_names:
        ramp(MDAC_channels[gate].voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

t0 = time.time()
#now do the real thing
do_multiple_gate_multiple_resonator_sweep(gate_channels, gate_voltages,gate_names,
                                 f_starts = fstarts, f_stops = fstops, f_names=f_names,
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=power, n_pts = n_pts, n_avg=n_avg, channel_name='S21')

print(" Time elapsed: %.4f" % (time.time()-t0))

### Single sweep with bot gate

In [ ]:
set_mdac_in_safemode()

In [ ]:
## BOTTOM GATE SWEEP WITH FIXED CUTTERs
power = -33
n_avgs = 400
n_pts = 100
v_cuts = np.arange(3,-3,-0.1)

for v_cut in v_cuts:
    exp = qc.new_experiment("Bot sweep: Vcut=%.3fV power=%d dbM navg=%d n_points=%d" %(v_cut, power, n_avgs, n_pts)  , sample_name="abs_w1_5_1_1")
    gate_channel = mdac.ch18
    gate_voltages = np.arange(-4,4,0.02)
    gate_name = 'V_bot_A'

    df = 0.0045e9

    #ramp back to the -4. 
    #print("ramping back ")
    #ramp(mdac.ch01.voltage, v_cut, gate_ramp_stepsize, gate_ramp_delay)
    #print("")
    #ramp(mdac.ch01.voltage, 1, gate_ramp_stepsize, gate_ramp_delay)
    #print("")
    #resonator A
    for vBot in Vbots:
        ramp(mdac.ch01.voltage, v_cut, gate_ramp_stepsize,gate_ramp_delay)
        f0 = 4.84725e9
        run_id=do_gate_resonator_sweep(gate_channel, gate_voltages,gate_name,
                                gate_ramp_stepsize = 2e-3,  #[V] previously was 1e-3
                                gate_ramp_delay = 0.003 , #[s]
                                power=power, n_pts = n_pts, f_start = f0-df, f_stop = f0+df, n_avg=n_avgs, channel_name='S12')


## Sweep of magnet at cutter gate

# Doing single point sweeps.


In [88]:
def setup_single_vna_point(frq, npts, power,  chan='S12', span=1, n_avg=10, rf_on = True):
    vna_channel = getattr(vna.channels, chan)
    
    vna_channel.power(power)
    vna_channel.format('Complex')
    vna_channel.center(frq)
    vna_channel.span(span)
    vna_channel.npts(npts)
    vna_channel.avg(n_avg) #todo check which is faster, avg 1 or more points.. #seems like avg is faster
    vna_channel.autoscale()
    if(rf_on):
        vna.rf_on()
    else:
        vna.rf_off()
    
def measure_complex_point_vna(chan='S12'):
    vna_channel = getattr(vna.channels, chan)
    S12 = vna_channel.trace.get()
    
    return np.mean(S12, dtype=complex)

    
def measure_mag_phase_point_vna(chan='S12'):
    vna_channel = getattr(vna.channels, chan)
    mag, phase = vna_channel.trace_mag_phase()
    
    return np.mean(mag), np.mean(phase)

def measure_and_process_point_vna(chan='S12'):
    """
    Measures processes a single point in vna and returns it as qcodes result object
    with param names:
    phase
    mag
    magnitude_dB
    """
    mag, phase = measure_mag_phase_point_vna(chan=channel_name)
    mag_db = 20*np.log10(mag) #convert to dB
    result = [ 
        ('phase', phase),
        ('mag', mag),
        ('magnitude_dB',mag_db )]
    return result


def do_gate_sweep_single_frequency(gate_channel, gate_voltages,gate_name='V_g',
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=-30, n_pts = 100, f = 4.8523e9, n_avg=100, channel_name='S12'):

    """
    performs a gate sweep at a single frequency. Can also sweep multiple parameters settiong the same voltages 
    
    """
    if not isinstance(gate_channel, list):
        gate_channel = [gate_channel] #allow for multple gates to be swept at once. 
        if not isinstance(gate_name, list):
            gate_name = [gate_name]
        else:
            raise Exception('Either give both gate channel and gate name as a list or both as a value..')
    
    #check the gate voltage
    for chan in gate_channel:
        print(chan,chan.voltage())
        
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    print("STARTING TEMPERATURE: %.3f" % temperature)
    
    # let's set the trace properties. 
    setup_single_vna_point( f,n_pts, power, n_avg=n_avg, chan=channel_name)
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency_readout', unit='Hz',
                                   paramtype='numeric')
    long_gate_name = ''
    for name in gate_name:
        long_gate_name += name + '_' #make one variable out of multiple gates snce they are swept together
    
    meas.register_custom_parameter(long_gate_name, unit='V')
    
    meas.register_custom_parameter('power', unit='dBm')
    meas.register_custom_parameter('n_pts', unit='')
    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=[long_gate_name],
                                   paramtype='numeric')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=[long_gate_name],
                                   paramtype='numeric')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=[long_gate_name],
                                   paramtype='numeric')
    
    with meas.run() as datasaver:
        for gate_voltage in gate_voltages:
            
            result=[]
            
            #ramp the gate voltage
            for i in range(len(gate_channel)):
                print("Ramping %s to %.2e" % (gate_name[i], gate_voltage))
                ramp(gate_channel[i].voltage, gate_voltage, gate_ramp_stepsize, gate_ramp_delay )
                print("Done ramping")
            
            result.append((long_gate_name, gate_voltage))
            
            vna_result= measure_and_process_point_vna(chan=channel_name)
            result.extend(vna_result)
            
            result.extend([
                ('temperature', temperature),
                ('power', power),
                ('frequency_readout', f),
                ('n_pts', n_pts)])
        
            datasaver.add_result(*result)
    
    return datasaver.run_id



def do_field_sweep_single_frequency(B_ys, power=-30, n_pts = 100, f = 4.8523e9, n_avg=100, channel_name='S12'):

    """
    performs a field sweep at a single frequency. Can also sweep multiple parameters settiong the same voltages 
    
    """
   
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    setup_single_vna_point( f,n_pts, power, n_avg=n_avg, chan=channel_name)
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency_readout', unit='Hz',
                                   paramtype='numeric')
    meas.register_custom_parameter('B_y', unit='T', paramtype='numeric')
    
    meas.register_custom_parameter('power', unit='dBm')
    meas.register_custom_parameter('n_pts', unit='')
    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=['B_y'],
                                   paramtype='numeric')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=['B_y'],
                                   paramtype='numeric')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=['B_y'],
                                   paramtype='numeric')
    
    with meas.run() as datasaver:
        for field in B_ys:      
   
            #ramp the field
            print("Setting field to", field)
            set_yoko_field(field)
            
            result= measure_and_process_point_vna(chan=channel_name)
            
            result.extend([
                ('B_y', field),
                ('temperature', temperature),
                ('power', power),
                ('frequency_readout', f),
                ('n_pts', n_pts)])
        
            datasaver.add_result(*result)
    
    return datasaver.run_id



def do_3D_field_sweep_single_frequency(B_field_mags, phi, theta, power=-30, n_pts = 100, f = 4.8523e9, n_avg=100, channel_name='S12'):

    """
    performs a field sweep at a single frequency.
    But now it sweeps the magnitude of the field at a certain phi and theta
    
    """
   
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    # let's set the trace properties. 
    setup_single_vna_point( f,n_pts, power, n_avg=n_avg, chan=channel_name)
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency_readout', unit='Hz',
                                   paramtype='numeric')
    meas.register_custom_parameter('B_field', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_x', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_y', unit='T', paramtype='numeric')
    meas.register_custom_parameter('B_z', unit='T', paramtype='numeric')
    
    meas.register_custom_parameter('power', unit='dBm')
    meas.register_custom_parameter('n_pts', unit='')
    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=['B_field'],
                                   paramtype='numeric')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=['B_field'],
                                   paramtype='numeric')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=['B_field'],
                                   paramtype='numeric')
    
    with meas.run() as datasaver:
        for field in B_field_mags:
            
            #ramp the field
            print("Setting field (magntidue) to", field)
            x, y, z = yoko_magnet.transform_spherical_to_cartesian(field, phi=phi, theta=theta)
            print("Setting x, y, z to %.3e, %.3e, %.3e" %(x,y,z))
            yoko_magnet.x(x)
            yoko_magnet.y(y)
            yoko_magnet.z(z)
            
            result= measure_and_process_point_vna(chan=channel_name)
            
            result.extend([
                ('B_field', field),
                ('B_x', x),
                ('B_y', y),
                ('B_z', z),
                ('temperature', temperature),
                ('power', power),
                ('frequency_readout', f),
                ('n_pts', n_pts)])
        
            datasaver.add_result(*result)
    
    return datasaver.run_id

In [89]:
def do_gate_sweep_multiple_frequency(gate_channel, gate_voltages,gate_name='V_g',
                                     fs = [4.8523e9], f_names=['A'],
                            gate_ramp_stepsize = 1e-3,  #[V] previously was 1e-3
                            gate_ramp_delay = 0.01 , #[s]
                            power=-30, n_pts = 100, n_avg=100, channel_name='S12'):

    """
    performs a gate sweep at a MULTIPLE frequencies. Can also sweep multiple parameters settiong the same voltages 
    
    """
    vna_channel = getattr(vna.channels, channel_name)
    
    if not isinstance(gate_channel, list):
        gate_channel = [gate_channel] #allow for multple gates to be swept at once. 
        if not isinstance(gate_name, list):
            gate_name = [gate_name]
        else:
            raise Exception('Either give both gate channel and gate name as a list or both as a value..')
    
    #check the gate voltage
    for chan in gate_channel:
        print(chan,chan.voltage())
        
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    print("STARTING TEMPERATURE: %.3f" % temperature)
    
    # let's set the trace properties. 
    setup_single_vna_point( fs[0],n_pts, power, n_avg=n_avg, chan=channel_name)
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency_readout', unit='Hz',
                                   paramtype='numeric')
    long_gate_name = ''
    for name in gate_name:
        long_gate_name += name + '_' #make one variable out of multiple gates snce they are swept together
    
    meas.register_custom_parameter(long_gate_name, unit='V')
    
    meas.register_custom_parameter('power', unit='dBm')
    meas.register_custom_parameter('n_pts', unit='')
    for i in range(len(fs)):
        meas.register_custom_parameter('frequency_'+f_names[i], unit='Hz',
                                   paramtype='numeric')
        meas.register_custom_parameter('mag_'+f_names[i], unit='V', 
                                       setpoints=[long_gate_name],
                                       paramtype='numeric')
        meas.register_custom_parameter('phase_'+f_names[i], unit='V', 
                                       setpoints=[long_gate_name],
                                       paramtype='numeric')
        meas.register_custom_parameter('magnitude_dB_'+f_names[i], unit='dB', 
                                       setpoints=[long_gate_name],
                                       paramtype='numeric')
    
    with meas.run() as datasaver:
        for gate_voltage in gate_voltages:
            
            result=[]
            
            #ramp the gate voltage
            for i in range(len(gate_channel)):
                print("Ramping %s to %.2e" % (gate_name[i], gate_voltage))
                ramp(gate_channel[i].voltage, gate_voltage, gate_ramp_stepsize, gate_ramp_delay )
                print("Done ramping")
            
            result.append((long_gate_name, gate_voltage))
            
            phases= np.array(len(fs))
            mags = np.array(len(fs))
            magnitude_dbs = np.array(len(fs)) 
            for i in range(len(fs)):
                vna_channel.center(fs[i])
                mag, phase = measure_mag_phase_point_vna(chan=channel_name)
                mag_db = 20*np.log10(mag) #convert to dB
                
                result.extend([
                    ('magnitude_dB_'+f_names[i],mag_db ),
                    ('phase_'+f_names[i], phase),
                    ('mag_'+f_names[i], mag),
                ])
            
            result.extend([
                ('temperature', temperature),
                ('power', power),
                ('frequency_readout', f),
                ('n_pts', n_pts)])
        
            datasaver.add_result(*result)
    
    return datasaver.run_id




In [93]:
B_ys = np.arange(0,0.7e-3,0.2e-6)
res_freqs = np.array([ 4.6754e9 ,5.6459e9, 3.6348e9,3.9983e9])
df = -1e6
power=-30
for f0 in res_freqs:
    #take a trace around f first after rampingto check where we are
    print("Taking single sweep")
    t0=time.time()
    do_single_sweep(n_pts = 500, f_start = f0-5e6, f_stop = f0+5e6, power=power, n_avg=100, channel_name='S21')
    print("Taking trace took", time.time()-t0)
    do_field_sweep_single_frequency(B_ys, power=power, n_pts = 200,f=f0+df ,n_avg=200, channel_name='S21')

Taking single sweep
Starting experimental run with id: 5
Taking trace took 6.24405574798584
Instrument T_probe already exists.
Path of network drive(set this manually dear USER):  \\MININT-ETA6HHA\Desktop\BF Log Files\
Starting experimental run with id: 6
Setting field to 0.0
Setting field to 2e-07
Setting field to 4e-07
Setting field to 6e-07
Setting field to 8e-07
Setting field to 1e-06
Setting field to 1.2e-06
Setting field to 1.4e-06
Setting field to 1.6e-06
Setting field to 1.8e-06
Setting field to 2e-06
Setting field to 2.2e-06
Setting field to 2.4e-06
Setting field to 2.5999999999999997e-06
Setting field to 2.8e-06
Setting field to 3e-06
Setting field to 3.2e-06
Setting field to 3.3999999999999996e-06
Setting field to 3.6e-06
Setting field to 3.8e-06
Setting field to 4e-06
Setting field to 4.2e-06
Setting field to 4.4e-06
Setting field to 4.6e-06
Setting field to 4.8e-06
Setting field to 4.9999999999999996e-06
Setting field to 5.199999999999999e-06
Setting field to 5.4e-06
Setti

Setting field to 5.3599999999999995e-05
Setting field to 5.38e-05
Setting field to 5.4e-05
Setting field to 5.4199999999999996e-05
Setting field to 5.4399999999999994e-05
Setting field to 5.46e-05
Setting field to 5.48e-05
Setting field to 5.4999999999999995e-05
Setting field to 5.52e-05
Setting field to 5.54e-05
Setting field to 5.5599999999999996e-05
Setting field to 5.5799999999999994e-05
Setting field to 5.6e-05
Setting field to 5.62e-05
Setting field to 5.6399999999999995e-05
Setting field to 5.66e-05
Setting field to 5.68e-05
Setting field to 5.6999999999999996e-05
Setting field to 5.7199999999999994e-05
Setting field to 5.74e-05
Setting field to 5.76e-05
Setting field to 5.7799999999999995e-05
Setting field to 5.8e-05
Setting field to 5.82e-05
Setting field to 5.8399999999999997e-05
Setting field to 5.8599999999999995e-05
Setting field to 5.88e-05
Setting field to 5.9e-05
Setting field to 5.9199999999999996e-05
Setting field to 5.94e-05
Setting field to 5.96e-05
Setting field to

Setting field to 0.00010839999999999999
Setting field to 0.0001086
Setting field to 0.00010879999999999999
Setting field to 0.00010899999999999999
Setting field to 0.0001092
Setting field to 0.00010939999999999999
Setting field to 0.0001096
Setting field to 0.0001098
Setting field to 0.00010999999999999999
Setting field to 0.0001102
Setting field to 0.0001104
Setting field to 0.00011059999999999999
Setting field to 0.0001108
Setting field to 0.000111
Setting field to 0.00011119999999999999
Setting field to 0.0001114
Setting field to 0.00011159999999999999
Setting field to 0.0001118
Setting field to 0.000112
Setting field to 0.00011219999999999999
Setting field to 0.0001124
Setting field to 0.0001126
Setting field to 0.00011279999999999999
Setting field to 0.000113
Setting field to 0.0001132
Setting field to 0.00011339999999999999
Setting field to 0.0001136
Setting field to 0.0001138
Setting field to 0.00011399999999999999
Setting field to 0.0001142
Setting field to 0.000114399999999999

Setting field to 0.00016219999999999999
Setting field to 0.0001624
Setting field to 0.0001626
Setting field to 0.0001628
Setting field to 0.000163
Setting field to 0.00016319999999999998
Setting field to 0.0001634
Setting field to 0.0001636
Setting field to 0.0001638
Setting field to 0.000164
Setting field to 0.00016419999999999998
Setting field to 0.00016439999999999998
Setting field to 0.0001646
Setting field to 0.0001648
Setting field to 0.000165
Setting field to 0.0001652
Setting field to 0.00016539999999999998
Setting field to 0.0001656
Setting field to 0.0001658
Setting field to 0.000166
Setting field to 0.0001662
Setting field to 0.00016639999999999998
Setting field to 0.00016659999999999998
Setting field to 0.0001668
Setting field to 0.000167
Setting field to 0.0001672
Setting field to 0.0001674
Setting field to 0.00016759999999999998
Setting field to 0.00016779999999999999
Setting field to 0.000168
Setting field to 0.0001682
Setting field to 0.0001684
Setting field to 0.000168

Setting field to 0.000215
Setting field to 0.0002152
Setting field to 0.00021539999999999998
Setting field to 0.00021559999999999998
Setting field to 0.0002158
Setting field to 0.000216
Setting field to 0.0002162
Setting field to 0.0002164
Setting field to 0.00021659999999999998
Setting field to 0.00021679999999999998
Setting field to 0.000217
Setting field to 0.0002172
Setting field to 0.0002174
Setting field to 0.00021759999999999998
Setting field to 0.00021779999999999998
Setting field to 0.00021799999999999999
Setting field to 0.0002182
Setting field to 0.0002184
Setting field to 0.0002186
Setting field to 0.00021879999999999998
Setting field to 0.00021899999999999998
Setting field to 0.0002192
Setting field to 0.0002194
Setting field to 0.0002196
Setting field to 0.0002198
Setting field to 0.00021999999999999998
Setting field to 0.00022019999999999999
Setting field to 0.0002204
Setting field to 0.0002206
Setting field to 0.0002208
Setting field to 0.00022099999999999998
Setting fi

...

Setting field to 0.000321
Setting field to 0.0003212
Setting field to 0.0003214
Setting field to 0.0003216
Setting field to 0.00032179999999999996
Setting field to 0.00032199999999999997
Setting field to 0.00032219999999999997
Setting field to 0.0003224
Setting field to 0.0003226
Setting field to 0.0003228
Setting field to 0.000323
Setting field to 0.0003232
Setting field to 0.0003234
Setting field to 0.0003236
Setting field to 0.0003238
Setting field to 0.00032399999999999996
Setting field to 0.00032419999999999997
Setting field to 0.00032439999999999997
Setting field to 0.0003246
Setting field to 0.0003248
Setting field to 0.000325
Setting field to 0.0003252
Setting field to 0.0003254
Setting field to 0.0003256
Setting field to 0.0003258
Setting field to 0.000326
Setting field to 0.00032619999999999996
Setting field to 0.00032639999999999996
Setting field to 0.00032659999999999997
Setting field to 0.0003268
Setting field to 0.000327
Setting field to 0.0003272
Setting field to 0.00032

Setting field to 0.0003744
Setting field to 0.0003746
Setting field to 0.0003748
Setting field to 0.000375
Setting field to 0.00037519999999999996
Setting field to 0.00037539999999999996
Setting field to 0.00037559999999999997
Setting field to 0.0003758
Setting field to 0.000376
Setting field to 0.0003762
Setting field to 0.0003764
Setting field to 0.0003766
Setting field to 0.0003768
Setting field to 0.000377
Setting field to 0.0003772
Setting field to 0.00037739999999999996
Setting field to 0.00037759999999999996
Setting field to 0.00037779999999999997
Setting field to 0.00037799999999999997
Setting field to 0.0003782
Setting field to 0.0003784
Setting field to 0.0003786
Setting field to 0.0003788
Setting field to 0.000379
Setting field to 0.0003792
Setting field to 0.0003794
Setting field to 0.00037959999999999996
Setting field to 0.00037979999999999996
Setting field to 0.00037999999999999997
Setting field to 0.00038019999999999997
Setting field to 0.0003804
Setting field to 0.00038

Setting field to 0.00042679999999999997
Setting field to 0.00042699999999999997
Setting field to 0.0004272
Setting field to 0.0004274
Setting field to 0.0004276
Setting field to 0.0004278
Setting field to 0.000428
Setting field to 0.0004282
Setting field to 0.0004284
Setting field to 0.00042859999999999996
Setting field to 0.00042879999999999996
Setting field to 0.00042899999999999997
Setting field to 0.00042919999999999997
Setting field to 0.0004294
Setting field to 0.0004296
Setting field to 0.0004298
Setting field to 0.00043
Setting field to 0.0004302
Setting field to 0.0004304
Setting field to 0.0004306
Setting field to 0.00043079999999999995
Setting field to 0.00043099999999999996
Setting field to 0.00043119999999999996
Setting field to 0.00043139999999999997
Setting field to 0.0004316
Setting field to 0.0004318
Setting field to 0.000432
Setting field to 0.0004322
Setting field to 0.0004324
Setting field to 0.0004326
Setting field to 0.0004328
Setting field to 0.000432999999999999

Setting field to 0.00047819999999999997
Setting field to 0.0004784
Setting field to 0.0004786
Setting field to 0.0004788
Setting field to 0.000479
Setting field to 0.0004792
Setting field to 0.0004794
Setting field to 0.0004796
Setting field to 0.00047979999999999995
Setting field to 0.00047999999999999996
Setting field to 0.00048019999999999996
Setting field to 0.00048039999999999997
Setting field to 0.00048059999999999997
Setting field to 0.0004808
Setting field to 0.000481
Setting field to 0.0004812
Setting field to 0.0004814
Setting field to 0.0004816
Setting field to 0.0004818
Setting field to 0.00048199999999999995
Setting field to 0.00048219999999999996
Setting field to 0.00048239999999999996
Setting field to 0.00048259999999999997
Setting field to 0.00048279999999999997
Setting field to 0.000483
Setting field to 0.0004832
Setting field to 0.0004834
Setting field to 0.0004836
Setting field to 0.0004838
Setting field to 0.000484
Setting field to 0.00048419999999999995
Setting fie

KeyboardInterrupt: 

## Parallel Field sweep of resonators. 

In [ ]:
param = yoko_magnet.z
print(np.append(np.arange(0,2,0.02),np.arange(2,-0.02,-0.02)))

In [ ]:
f_readouts=  [3533000000.0, 4018000000.0, 4844500000.0, 5884000000.0]

In [ ]:
power = -27
f0s_sweep = f0s
df = 10e6
n_pts = 1000
n_avg = 1000
B_fields = np.arange(2.25,3.5,0.05)

for field in B_fields:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    print('T before ramping: ', temperature)
    yoko_magnet.z(field)
    temperature = T_probe.temperature()
    print('T after ramping: ', temperature)
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    print(' T before start', temperature)
    print(field)
    for i in range(len(f0s)):
        f = f0s_sweep[i]
        print("Taking single sweep")
        t0=time.time()
        exp = qc.new_experiment("freq sweep f0=%.5e df=%.3e power=%d n=%d x %d Bz=%.2e" %(f,df, power, n_pts, n_avg, field)  , sample_name="abs_w1_5_1_1")
        run_id = do_single_sweep(n_pts = n_pts, f_start = f-df, f_stop = f+df, power=power, n_avg=n_avg, channel_name='S21',
                                extra_params = [(yoko_magnet.z, field)])
        print("Taking trace took", time.time()-t0)
        dat = load_by_id(run_id)

        #now update the readout frequency
        freqs = np.squeeze(dat.get_data('frequency'))
        mag = np.squeeze(dat.get_data('magnitude_raw'))
        f0s_sweep[i] = freqs[np.argmin(mag)]
    print(f0s_sweep)
yoko_magnet.z(0)

In [ ]:
yoko_magnet.y()

## 0701 Power sweep single frequency vs flux

In [107]:
def do_field_power_sweep_single_frequency(B_ys, powers, n_pts = 100, f = 4.8523e9, n_avg=100, channel_name='S21',
                                         reduce_n_pts_during_sweep = True):

    """
    performs a field sweep at a single frequency. Can also sweep multiple parameters settiong the same voltages 
    
    """
   
    #first try to get temperature:
    T_probe = create_inst(Thermometer, 'T_probe', logfile_path=BF_PC_PATH, force_new_instance=True)
    temperature = T_probe.temperature()
    
    
    
    #now setup the qcodes measurement object
    meas = Measurement()
    meas.register_custom_parameter('temperature', unit='K', paramtype='numeric')
    #meas.register_parameter(temperature_sample, setpoints=['time',])
    meas.register_custom_parameter('frequency_readout', unit='Hz',
                                   paramtype='numeric')
    meas.register_custom_parameter('B_y', unit='T', paramtype='numeric')
    
    meas.register_custom_parameter('power', unit='dBm')
    meas.register_custom_parameter('n_pts', unit='')
    meas.register_custom_parameter('n_avg', unit='')
    meas.register_custom_parameter('mag', unit='V', 
                                   setpoints=['B_y', 'power'],
                                   paramtype='numeric')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=['B_y', 'power'],
                                   paramtype='numeric')
    meas.register_custom_parameter('magnitude_dB', unit='dB', 
                                   setpoints=['B_y', 'power'],
                                   paramtype='numeric')
    
    with meas.run() as datasaver:
        for power in powers:
            # let's set the trace properties. 
            
            setup_single_vna_point( f,n_pts, power, n_avg=n_avg, chan=channel_name)
            
            for field in B_ys:      

                #ramp the field
                print("Setting field to", field)
                set_yoko_field(field)

                result= measure_and_process_point_vna(chan=channel_name)

                result.extend([
                    ('B_y', field),
                    ('temperature', temperature),
                    ('power', power),
                    ('frequency_readout', f),
                    ('n_pts', n_pts),
                    ('n_avg', n_avg)])

                datasaver.add_result(*result)
            if reduce_n_pts_during_sweep:
                    n_pts-= 5
                    print('Reduced npts to %d' % n_pts)
    return datasaver.run_id


In [128]:
# define params:
res_freqs = np.array([ 4.6754e9 ,5.6459e9, 3.6348e9,3.9983e9])
f0 = res_freqs[0]
f_readout = f0-0.5e6
n_avg = 300
n_pts = 300
channel = 'S21'

#set 2D sweep params
B_ys = np.arange(0,0.5e-3,2e-6)
#B_ys = np.zeros(1000)
powers = np.arange(-45,10,3)

take_trace_at_beginning =True

# do a sweep to see the readout point
if take_trace_at_beginning:
    do_single_sweep(n_pts = 500, f_start = f_readout-5e6, f_stop = f_readout+5e6, power=powers[0], n_avg=400, channel_name='S21')
   
exp = qc.new_experiment("Power sweep versus flux. n=%dx%d fr=%.5e" %(n_pts, n_avg, f_readout)  , sample_name="abs_w1_3_RES")

do_field_power_sweep_single_frequency(B_ys, powers, n_pts = n_pts, f = f_readout, n_avg=n_avg, channel_name=channel)

Starting experimental run with id: 57
Instrument T_probe already exists.
Path of network drive(set this manually dear USER):  \\MININT-ETA6HHA\Desktop\BF Log Files\
Starting experimental run with id: 58
Setting field to 0.0
Setting field to 2e-06
Setting field to 4e-06
Setting field to 6e-06
Setting field to 8e-06
Setting field to 9.999999999999999e-06
Setting field to 1.2e-05
Setting field to 1.4e-05
Setting field to 1.6e-05
Setting field to 1.8e-05
Setting field to 1.9999999999999998e-05
Setting field to 2.2e-05
Setting field to 2.4e-05
Setting field to 2.6e-05
Setting field to 2.8e-05
Setting field to 2.9999999999999997e-05
Setting field to 3.2e-05
Setting field to 3.4e-05
Setting field to 3.6e-05
Setting field to 3.7999999999999995e-05
Setting field to 3.9999999999999996e-05
Setting field to 4.2e-05
Setting field to 4.4e-05
Setting field to 4.6e-05
Setting field to 4.8e-05
Setting field to 4.9999999999999996e-05
Setting field to 5.2e-05
Setting field to 5.4e-05
Setting field to 5.6

Setting field to 3.7999999999999995e-05
Setting field to 3.9999999999999996e-05
Setting field to 4.2e-05
Setting field to 4.4e-05
Setting field to 4.6e-05
Setting field to 4.8e-05
Setting field to 4.9999999999999996e-05
Setting field to 5.2e-05
Setting field to 5.4e-05
Setting field to 5.6e-05
Setting field to 5.8e-05
Setting field to 5.9999999999999995e-05
Setting field to 6.2e-05
Setting field to 6.4e-05
Setting field to 6.599999999999999e-05
Setting field to 6.8e-05
Setting field to 7e-05
Setting field to 7.2e-05
Setting field to 7.4e-05
Setting field to 7.599999999999999e-05
Setting field to 7.8e-05
Setting field to 7.999999999999999e-05
Setting field to 8.2e-05
Setting field to 8.4e-05
Setting field to 8.599999999999999e-05
Setting field to 8.8e-05
Setting field to 8.999999999999999e-05
Setting field to 9.2e-05
Setting field to 9.4e-05
Setting field to 9.6e-05
Setting field to 9.8e-05
Setting field to 9.999999999999999e-05
Setting field to 0.000102
Setting field to 0.000104
Settin

Setting field to 8.599999999999999e-05
Setting field to 8.8e-05
Setting field to 8.999999999999999e-05
Setting field to 9.2e-05
Setting field to 9.4e-05
Setting field to 9.6e-05
Setting field to 9.8e-05
Setting field to 9.999999999999999e-05
Setting field to 0.000102
Setting field to 0.000104
Setting field to 0.000106
Setting field to 0.000108
Setting field to 0.00010999999999999999
Setting field to 0.000112
Setting field to 0.00011399999999999999
Setting field to 0.000116
Setting field to 0.000118
Setting field to 0.00011999999999999999
Setting field to 0.000122
Setting field to 0.000124
Setting field to 0.000126
Setting field to 0.000128
Setting field to 0.00013
Setting field to 0.00013199999999999998
Setting field to 0.000134
Setting field to 0.000136
Setting field to 0.000138
Setting field to 0.00014
Setting field to 0.00014199999999999998
Setting field to 0.000144
Setting field to 0.000146
Setting field to 0.000148
Setting field to 0.00015
Setting field to 0.00015199999999999998
S

Setting field to 0.000134
Setting field to 0.000136
Setting field to 0.000138
Setting field to 0.00014
Setting field to 0.00014199999999999998
Setting field to 0.000144
Setting field to 0.000146
Setting field to 0.000148
Setting field to 0.00015
Setting field to 0.00015199999999999998
Setting field to 0.000154
Setting field to 0.000156
Setting field to 0.000158
Setting field to 0.00015999999999999999
Setting field to 0.00016199999999999998
Setting field to 0.000164
Setting field to 0.000166
Setting field to 0.000168
Setting field to 0.00016999999999999999
Setting field to 0.00017199999999999998
Setting field to 0.000174
Setting field to 0.000176
Setting field to 0.000178
Setting field to 0.00017999999999999998
Setting field to 0.00018199999999999998
Setting field to 0.000184
Setting field to 0.000186
Setting field to 0.000188
Setting field to 0.00018999999999999998
Setting field to 0.000192
Setting field to 0.000194
Setting field to 0.000196
Setting field to 0.000198
Setting field to 0

Setting field to 0.00018199999999999998
Setting field to 0.000184
Setting field to 0.000186
Setting field to 0.000188
Setting field to 0.00018999999999999998
Setting field to 0.000192
Setting field to 0.000194
Setting field to 0.000196
Setting field to 0.000198
Setting field to 0.00019999999999999998
Setting field to 0.000202
Setting field to 0.000204
Setting field to 0.000206
Setting field to 0.000208
Setting field to 0.00020999999999999998
Setting field to 0.000212
Setting field to 0.000214
Setting field to 0.000216
Setting field to 0.00021799999999999999
Setting field to 0.00021999999999999998
Setting field to 0.000222
Setting field to 0.000224
Setting field to 0.000226
Setting field to 0.00022799999999999999
Setting field to 0.00022999999999999998
Setting field to 0.000232
Setting field to 0.000234
Setting field to 0.000236
Setting field to 0.00023799999999999998
Setting field to 0.00023999999999999998
Setting field to 0.000242
Setting field to 0.000244
Setting field to 0.000245999

...

Setting field to 0.000156
Setting field to 0.000158
Setting field to 0.00015999999999999999
Setting field to 0.00016199999999999998
Setting field to 0.000164
Setting field to 0.000166
Setting field to 0.000168
Setting field to 0.00016999999999999999
Setting field to 0.00017199999999999998
Setting field to 0.000174
Setting field to 0.000176
Setting field to 0.000178
Setting field to 0.00017999999999999998
Setting field to 0.00018199999999999998
Setting field to 0.000184
Setting field to 0.000186
Setting field to 0.000188
Setting field to 0.00018999999999999998
Setting field to 0.000192
Setting field to 0.000194
Setting field to 0.000196
Setting field to 0.000198
Setting field to 0.00019999999999999998
Setting field to 0.000202
Setting field to 0.000204
Setting field to 0.000206
Setting field to 0.000208
Setting field to 0.00020999999999999998
Setting field to 0.000212
Setting field to 0.000214
Setting field to 0.000216
Setting field to 0.00021799999999999999
Setting field to 0.000219999

Setting field to 0.000202
Setting field to 0.000204
Setting field to 0.000206
Setting field to 0.000208
Setting field to 0.00020999999999999998
Setting field to 0.000212
Setting field to 0.000214
Setting field to 0.000216
Setting field to 0.00021799999999999999
Setting field to 0.00021999999999999998
Setting field to 0.000222
Setting field to 0.000224
Setting field to 0.000226
Setting field to 0.00022799999999999999
Setting field to 0.00022999999999999998
Setting field to 0.000232
Setting field to 0.000234
Setting field to 0.000236
Setting field to 0.00023799999999999998
Setting field to 0.00023999999999999998
Setting field to 0.000242
Setting field to 0.000244
Setting field to 0.00024599999999999996
Setting field to 0.000248
Setting field to 0.00025
Setting field to 0.000252
Setting field to 0.000254
Setting field to 0.000256
Setting field to 0.000258
Setting field to 0.00026
Setting field to 0.00026199999999999997
Setting field to 0.00026399999999999997
Setting field to 0.000266
Sett

Setting field to 0.000248
Setting field to 0.00025
Setting field to 0.000252
Setting field to 0.000254
Setting field to 0.000256
Setting field to 0.000258
Setting field to 0.00026
Setting field to 0.00026199999999999997
Setting field to 0.00026399999999999997
Setting field to 0.000266
Setting field to 0.000268
Setting field to 0.00027
Setting field to 0.000272
Setting field to 0.000274
Setting field to 0.000276
Setting field to 0.000278
Setting field to 0.00028
Setting field to 0.00028199999999999997
Setting field to 0.00028399999999999996
Setting field to 0.000286
Setting field to 0.000288
Setting field to 0.00029
Setting field to 0.000292
Setting field to 0.000294
Setting field to 0.000296
Setting field to 0.000298
Setting field to 0.0003
Setting field to 0.00030199999999999997
Setting field to 0.00030399999999999996
Setting field to 0.000306
Setting field to 0.000308
Setting field to 0.00031
Setting field to 0.000312
Setting field to 0.000314
Setting field to 0.000316
Setting field 

Setting field to 0.000298
Setting field to 0.0003
Setting field to 0.00030199999999999997
Setting field to 0.00030399999999999996
Setting field to 0.000306
Setting field to 0.000308
Setting field to 0.00031
Setting field to 0.000312
Setting field to 0.000314
Setting field to 0.000316
Setting field to 0.000318
Setting field to 0.00031999999999999997
Setting field to 0.00032199999999999997
Setting field to 0.00032399999999999996
Setting field to 0.000326
Setting field to 0.000328
Setting field to 0.00033
Setting field to 0.000332
Setting field to 0.000334
Setting field to 0.000336
Setting field to 0.000338
Setting field to 0.00033999999999999997
Setting field to 0.00034199999999999996
Setting field to 0.00034399999999999996
Setting field to 0.000346
Setting field to 0.000348
Setting field to 0.00035
Setting field to 0.000352
Setting field to 0.000354
Setting field to 0.000356
Setting field to 0.000358
Setting field to 0.00035999999999999997
Setting field to 0.00036199999999999996
Setting

58

In [110]:
# define params:
res_freqs = np.array([ 4.6754e9 ,5.6459e9, 3.6348e9,3.9983e9])
f0 = res_freqs[2]
f_readout = f0-0.5e6
n_avg = 200
n_pts = 200
channel = 'S21'

#set 2D sweep params
B_ys = np.arange(0,1e-3,2e-6)
#B_ys = np.zeros(1000)
powers = np.arange(-35,20,5)

take_trace_at_beginning =True

# do a sweep to see the readout point
if take_trace_at_beginning:
    do_single_sweep(n_pts = 500, f_start = f_readout-5e6, f_stop = f_readout+5e6, power=powers[0], n_avg=400, channel_name='S21')
   
exp = qc.new_experiment("Power sweep versus flux. n=%dx%d fr=%.5e" %(n_pts, n_avg, f_readout)  , sample_name="abs_w1_3_RES")

do_field_power_sweep_single_frequency(B_ys, powers, n_pts = n_pts, f = f_readout, n_avg=n_avg, channel_name=channel)

Starting experimental run with id: 30
Instrument T_probe already exists.
Path of network drive(set this manually dear USER):  \\MININT-ETA6HHA\Desktop\BF Log Files\
Starting experimental run with id: 31
Setting field to 0.0
Setting field to 2e-06
Setting field to 4e-06
Setting field to 6e-06
Setting field to 8e-06
Setting field to 9.999999999999999e-06
Setting field to 1.2e-05
Setting field to 1.4e-05
Setting field to 1.6e-05
Setting field to 1.8e-05
Setting field to 1.9999999999999998e-05
Setting field to 2.2e-05
Setting field to 2.4e-05
Setting field to 2.6e-05
Setting field to 2.8e-05
Setting field to 2.9999999999999997e-05
Setting field to 3.2e-05
Setting field to 3.4e-05
Setting field to 3.6e-05
Setting field to 3.7999999999999995e-05
Setting field to 3.9999999999999996e-05
Setting field to 4.2e-05
Setting field to 4.4e-05
Setting field to 4.6e-05
Setting field to 4.8e-05
Setting field to 4.9999999999999996e-05
Setting field to 5.2e-05
Setting field to 5.4e-05
Setting field to 5.6

Setting field to 0.000538
Setting field to 0.00054
Setting field to 0.000542
Setting field to 0.000544
Setting field to 0.0005459999999999999
Setting field to 0.000548
Setting field to 0.0005499999999999999
Setting field to 0.000552
Setting field to 0.000554
Setting field to 0.000556
Setting field to 0.000558
Setting field to 0.00056
Setting field to 0.000562
Setting field to 0.0005639999999999999
Setting field to 0.000566
Setting field to 0.0005679999999999999
Setting field to 0.00057
Setting field to 0.000572
Setting field to 0.000574
Setting field to 0.000576
Setting field to 0.000578
Setting field to 0.00058
Setting field to 0.0005819999999999999
Setting field to 0.000584
Setting field to 0.0005859999999999999
Setting field to 0.000588
Setting field to 0.0005899999999999999
Setting field to 0.000592
Setting field to 0.000594
Setting field to 0.000596
Setting field to 0.000598
Setting field to 0.0006
Setting field to 0.000602
Setting field to 0.0006039999999999999
Setting field to 0

Setting field to 8.599999999999999e-05
Setting field to 8.8e-05
Setting field to 8.999999999999999e-05
Setting field to 9.2e-05
Setting field to 9.4e-05
Setting field to 9.6e-05
Setting field to 9.8e-05
Setting field to 9.999999999999999e-05
Setting field to 0.000102
Setting field to 0.000104
Setting field to 0.000106
Setting field to 0.000108
Setting field to 0.00010999999999999999
Setting field to 0.000112
Setting field to 0.00011399999999999999
Setting field to 0.000116
Setting field to 0.000118
Setting field to 0.00011999999999999999
Setting field to 0.000122
Setting field to 0.000124
Setting field to 0.000126
Setting field to 0.000128
Setting field to 0.00013
Setting field to 0.00013199999999999998
Setting field to 0.000134
Setting field to 0.000136
Setting field to 0.000138
Setting field to 0.00014
Setting field to 0.00014199999999999998
Setting field to 0.000144
Setting field to 0.000146
Setting field to 0.000148
Setting field to 0.00015
Setting field to 0.00015199999999999998
S

Setting field to 0.000636
Setting field to 0.000638
Setting field to 0.0006399999999999999
Setting field to 0.000642
Setting field to 0.0006439999999999999
Setting field to 0.000646
Setting field to 0.0006479999999999999
Setting field to 0.00065
Setting field to 0.000652
Setting field to 0.000654
Setting field to 0.000656
Setting field to 0.000658
Setting field to 0.00066
Setting field to 0.0006619999999999999
Setting field to 0.000664
Setting field to 0.0006659999999999999
Setting field to 0.000668
Setting field to 0.00067
Setting field to 0.000672
Setting field to 0.000674
Setting field to 0.000676
Setting field to 0.000678
Setting field to 0.0006799999999999999
Setting field to 0.000682
Setting field to 0.0006839999999999999
Setting field to 0.000686
Setting field to 0.0006879999999999999
Setting field to 0.00069
Setting field to 0.000692
Setting field to 0.000694
Setting field to 0.000696
Setting field to 0.0006979999999999999
Setting field to 0.0007
Setting field to 0.000701999999

Setting field to 0.00018199999999999998
Setting field to 0.000184
Setting field to 0.000186
Setting field to 0.000188
Setting field to 0.00018999999999999998
Setting field to 0.000192
Setting field to 0.000194
Setting field to 0.000196
Setting field to 0.000198
Setting field to 0.00019999999999999998
Setting field to 0.000202
Setting field to 0.000204
Setting field to 0.000206
Setting field to 0.000208
Setting field to 0.00020999999999999998
Setting field to 0.000212
Setting field to 0.000214
Setting field to 0.000216
Setting field to 0.00021799999999999999
Setting field to 0.00021999999999999998
Setting field to 0.000222
Setting field to 0.000224
Setting field to 0.000226
Setting field to 0.00022799999999999999
Setting field to 0.00022999999999999998
Setting field to 0.000232
Setting field to 0.000234
Setting field to 0.000236
Setting field to 0.00023799999999999998
Setting field to 0.00023999999999999998
Setting field to 0.000242
Setting field to 0.000244
Setting field to 0.000245999

...

Setting field to 0.00030399999999999996
Setting field to 0.000306
Setting field to 0.000308
Setting field to 0.00031
Setting field to 0.000312
Setting field to 0.000314
Setting field to 0.000316
Setting field to 0.000318
Setting field to 0.00031999999999999997
Setting field to 0.00032199999999999997
Setting field to 0.00032399999999999996
Setting field to 0.000326
Setting field to 0.000328
Setting field to 0.00033
Setting field to 0.000332
Setting field to 0.000334
Setting field to 0.000336
Setting field to 0.000338
Setting field to 0.00033999999999999997
Setting field to 0.00034199999999999996
Setting field to 0.00034399999999999996
Setting field to 0.000346
Setting field to 0.000348
Setting field to 0.00035
Setting field to 0.000352
Setting field to 0.000354
Setting field to 0.000356
Setting field to 0.000358
Setting field to 0.00035999999999999997
Setting field to 0.00036199999999999996
Setting field to 0.00036399999999999996
Setting field to 0.000366
Setting field to 0.000368
Setti

Setting field to 0.000852
Setting field to 0.0008539999999999999
Setting field to 0.000856
Setting field to 0.0008579999999999999
Setting field to 0.00086
Setting field to 0.0008619999999999999
Setting field to 0.000864
Setting field to 0.0008659999999999999
Setting field to 0.000868
Setting field to 0.00087
Setting field to 0.0008719999999999999
Setting field to 0.000874
Setting field to 0.0008759999999999999
Setting field to 0.000878
Setting field to 0.0008799999999999999
Setting field to 0.000882
Setting field to 0.0008839999999999999
Setting field to 0.000886
Setting field to 0.000888
Setting field to 0.00089
Setting field to 0.000892
Setting field to 0.0008939999999999999
Setting field to 0.000896
Setting field to 0.0008979999999999999
Setting field to 0.0009
Setting field to 0.0009019999999999999
Setting field to 0.000904
Setting field to 0.000906
Setting field to 0.000908
Setting field to 0.00091
Setting field to 0.0009119999999999999
Setting field to 0.000914
Setting field to 0

Setting field to 0.00039999999999999996
Setting field to 0.00040199999999999996
Setting field to 0.000404
Setting field to 0.000406
Setting field to 0.000408
Setting field to 0.00041
Setting field to 0.000412
Setting field to 0.000414
Setting field to 0.000416
Setting field to 0.00041799999999999997
Setting field to 0.00041999999999999996
Setting field to 0.00042199999999999996
Setting field to 0.000424
Setting field to 0.000426
Setting field to 0.000428
Setting field to 0.00043
Setting field to 0.000432
Setting field to 0.000434
Setting field to 0.00043599999999999997
Setting field to 0.00043799999999999997
Setting field to 0.00043999999999999996
Setting field to 0.00044199999999999996
Setting field to 0.000444
Setting field to 0.000446
Setting field to 0.000448
Setting field to 0.00045
Setting field to 0.000452
Setting field to 0.000454
Setting field to 0.00045599999999999997
Setting field to 0.00045799999999999997
Setting field to 0.00045999999999999996
Setting field to 0.0004619999

Setting field to 0.000946
Setting field to 0.000948
Setting field to 0.00095
Setting field to 0.0009519999999999999
Setting field to 0.000954
Setting field to 0.0009559999999999999
Setting field to 0.000958
Setting field to 0.0009599999999999999
Setting field to 0.000962
Setting field to 0.0009639999999999999
Setting field to 0.000966
Setting field to 0.000968
Setting field to 0.0009699999999999999
Setting field to 0.000972
Setting field to 0.0009739999999999999
Setting field to 0.000976
Setting field to 0.000978
Setting field to 0.00098
Setting field to 0.000982
Setting field to 0.0009839999999999998
Setting field to 0.000986
Setting field to 0.000988
Setting field to 0.00099
Setting field to 0.000992
Setting field to 0.0009939999999999999
Setting field to 0.000996
Setting field to 0.000998
Setting field to 0.001
Reduced npts to 145


31